In [1]:
%pip install streamlit

Note: you may need to restart the kernel to use updated packages.


In [3]:
!streamlit run customer_segmentation_dashboard.py

Usage: streamlit run [OPTIONS] TARGET [ARGS]...
Try 'streamlit run --help' for help.

Error: Invalid value: File does not exist: customer_segmentation_dashboard.py


In [4]:
import streamlit as st
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt

In [5]:
df = pd.read_csv('shopping_behavior_updated.csv')
df.head()

,Customer ID,Age,Gender,Item Purchased,Category,Purchase Amount (USD),Location,Size,Color,Season,Review Rating,Subscription Status,Shipping Type,Discount Applied,Promo Code Used,Previous Purchases,Payment Method,Frequency of Purchases
0,1,55,Male,Blouse,Clothing,53,Kentucky,L,Gray,Winter,3.1,Yes,Express,Yes,Yes,14,Venmo,Fortnightly
1,2,19,Male,Sweater,Clothing,64,Maine,L,Maroon,Winter,3.1,Yes,Express,Yes,Yes,2,Cash,Fortnightly
2,3,50,Male,Jeans,Clothing,73,Massachusetts,S,Maroon,Spring,3.1,Yes,Free Shipping,Yes,Yes,23,Credit Card,Weekly
3,4,21,Male,Sandals,Footwear,90,Rhode Island,M,Maroon,Spring,3.5,Yes,Next Day Air,Yes,Yes,49,PayPal,Weekly
4,5,45,Male,Blouse,Clothing,49,Oregon,M,Turquoise,Spring,2.7,Yes,Free Shipping,Yes,Yes,31,PayPal,Annually


In [6]:
# Mapping for categorical frequencies to numerical values
frequency_mapping = {
    "Weekly": 52,
    "Fortnightly": 26,
    "Bi-Weekly": 24,
    "Monthly": 12,
    "Every 3 Months": 4,
    "Quarterly": 4,
    "Annually": 1
}

# Apply the mapping
df["Frequency of Purchases (Numeric)"] = df["Frequency of Purchases"].map(frequency_mapping)

# Drop the original categorical column
df.drop(columns=["Frequency of Purchases"], inplace=True)

# Display updated dataset
print(df.head())

   Customer ID  Age Gender Item Purchased  Category  Purchase Amount (USD)  \
0            1   55   Male         Blouse  Clothing                     53   
1            2   19   Male        Sweater  Clothing                     64   
2            3   50   Male          Jeans  Clothing                     73   
3            4   21   Male        Sandals  Footwear                     90   
4            5   45   Male         Blouse  Clothing                     49   

        Location Size      Color  Season  Review Rating Subscription Status  \
0       Kentucky    L       Gray  Winter            3.1                 Yes   
1          Maine    L     Maroon  Winter            3.1                 Yes   
2  Massachusetts    S     Maroon  Spring            3.1                 Yes   
3   Rhode Island    M     Maroon  Spring            3.5                 Yes   
4         Oregon    M  Turquoise  Spring            2.7                 Yes   

   Shipping Type Discount Applied Promo Code Used  Previ

In [ ]:
#%pip install sentence-transformers
#%pip install tf-keras
#%pip install scikit-learn

from sklearn.decomposition import PCA
from sentence_transformers import SentenceTransformer

# Pre-trained language model
model = SentenceTransformer('all-MiniLM-L6-v2')

df['Category_Embedding'] = df['Category'].apply(lambda x: model.encode(str(x)))
df['Item_Purchased_Embedding'] = df['Item Purchased'].apply(lambda x: model.encode(str(x)))

# Convert embeddings to NumPy arrays
category_embeddings = np.vstack(df['Category_Embedding'].values)
item_purchased_embeddings = np.vstack(df['Item_Purchased_Embedding'].values)

# Reduce dimensionality using PCA
pca = PCA(n_components=20)  
category_embeddings_pca = pca.fit_transform(category_embeddings)
item_purchased_embeddings_pca = pca.fit_transform(item_purchased_embeddings)

# Concatenate PCA-reduced embeddings with numerical data
numerical_features = ['Age', 'Frequency', 'Monetary', 'Engagement_Score', 'Recency']
from sklearn.preprocessing import StandardScaler

# Create the required columns
df['Recency'] = df['Previous Purchases']
df['Frequency'] = df['Frequency of Purchases (Numeric)']
df['Monetary'] = df['Purchase Amount (USD)']

# Ensure categorical "Subscription Status" is standardized
df["Subscription Status"] = df["Subscription Status"].str.lower().str.strip()

# Map Subscription Status: 1 for subscribed, 0 for non-subscribed
df["Subscription_Status_Binary"] = df["Subscription Status"].apply(lambda x: 1 if x == "subscribed" else 0)

# Compute Engagement (If Review Rating is available)
if "Review Rating" in df.columns:
    df["Engagement_Score"] = df["Review Rating"]
else:
    df["Engagement_Score"] = 0  # Default if missing

# Loyalty Score (Combination of Subscription & Purchase Frequency)
df["Loyalty_Score"] = df["Frequency of Purchases (Numeric)"] + df["Previous Purchases"] + (df["Subscription_Status_Binary"] * 10)


X_embeddings = np.hstack([df[numerical_features].values, category_embeddings_pca, item_purchased_embeddings_pca])

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [ ]:
#%pip install scikit-learn
from sklearn.cluster import KMeans

# Apply K-Means
kmeans = KMeans(n_clusters=4, random_state=42, n_init=10)
df['KMeans_LLM_Cluster'] = kmeans.fit_predict(X_embeddings)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [16]:
from sklearn.metrics import silhouette_score

# Silhouette Score for K-Means
silhouette_score_kmeans = silhouette_score(X_embeddings, df['KMeans_LLM_Cluster'])

# Final clustering results
clustering_results = {
    "Number of Clusters": 4,
    "Silhouette Score": silhouette_score_kmeans,
    "Total Features After PCA + Numerical Data": X_embeddings.shape[1]
}

# Convert results to DataFrame and display
clustering_results_df = pd.DataFrame([clustering_results])
print(clustering_results_df)

   Number of Clusters  Silhouette Score  \
0                   4          0.248385   

   Total Features After PCA + Numerical Data  
0                                         45  


In [17]:
import seaborn as sns

plt.figure(figsize=(12, 6))
sns.countplot(x="KMeans_LLM_Cluster", data=df, order=df["KMeans_LLM_Cluster"].value_counts().index)
plt.title("Customer Segmentation Based on K-Means with LLM Embeddings")
plt.xticks(rotation=45)
plt.show()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/var/folders/4g/7cwxt52n09sb7vzyh6tzj3w00000gn/T/ipykernel_63535/1000046078.py:7: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [18]:
# Define clusters & metrics
cluster_means_llm = df.groupby("KMeans_LLM_Cluster")[numerical_features].mean()
print(cluster_means_llm)

# Number of variables
categories = list(cluster_means_llm.columns)
N = len(categories)

# Convert to radar chart format
values = cluster_means_llm.values
cluster_names = [f'Cluster {i}' for i in range(len(cluster_means_llm))]
angles = np.linspace(0, 2 * np.pi, N, endpoint=False).tolist()

# Plot Radar Chart
fig, ax = plt.subplots(figsize=(8, 8), subplot_kw=dict(polar=True))
for i in range(len(values)):
    ax.plot(angles, values[i], linewidth=1, linestyle='solid', label=cluster_names[i])
    ax.fill(angles, values[i], alpha=0.2)

# Configure labels
ax.set_xticks(angles)
ax.set_xticklabels(categories)
plt.title("Cluster Comparison via Radar Chart (LLM Embeddings)")
plt.legend()

plt.show()

                          Age  Frequency   Monetary  Engagement_Score  \
KMeans_LLM_Cluster                                                      
0                   57.782988  11.198473  79.095965          3.784951   
1                   44.297243  11.627438  36.936113          3.715736   
2                   44.657407  51.951852  58.972222          3.757778   
3                   30.224895  11.957113  77.176778          3.765167   

                      Recency  
KMeans_LLM_Cluster             
0                   26.111232  
1                   25.083389  
2                   25.779630  
3                   24.798117  


/var/folders/4g/7cwxt52n09sb7vzyh6tzj3w00000gn/T/ipykernel_63535/2402893559.py:26: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [19]:
from sentence_transformers import SentenceTransformer

# Sentence embedding model 
embedder = SentenceTransformer("all-MiniLM-L6-v2")

# Categorical/text columns to convert into embeddings
text_columns = ["Category", "Subscription Status", "Payment Method"]

# Categorical values into embeddings
def encode_text_features(df, columns):
    text_data = df[columns].astype(str).agg(" ".join, axis=1)  # Concatenate selected text features
    embeddings = embedder.encode(text_data, convert_to_numpy=True)  # Generate text embeddings
    return embeddings

# Text embeddings
text_embeddings = encode_text_features(df, text_columns)

# Stack with numerical features for clustering
numerical_features = ["Recency", "Frequency", "Monetary", "Engagement_Score"]
X_combined = np.hstack((df[numerical_features].values, text_embeddings))

In [20]:
from sklearn.metrics import silhouette_score, davies_bouldin_score

# Determine the optimal number of clusters (Elbow Method)
inertia = []
k_range = range(2, 11)

for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(X_combined)
    inertia.append(kmeans.inertia_)

# Plot the Elbow Method
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 5))
plt.plot(k_range, inertia, marker="o")
plt.xlabel("Number of Clusters (k)")
plt.ylabel("Inertia (WCSS)")
plt.title("Elbow Method for LLM + K-Means Clustering")
plt.show()

# Choose optimal number of clusters
optimal_k_llm = 4  # Adjust based on elbow method

# Apply K-Means with chosen k
kmeans_llm = KMeans(n_clusters=optimal_k_llm, random_state=42, n_init=10)
df["LLM_KMeans_Cluster"] = kmeans_llm.fit_predict(X_combined)

# Evaluate clustering quality
silhouette_llm = silhouette_score(X_combined, df["LLM_KMeans_Cluster"])
davies_bouldin_llm = davies_bouldin_score(X_combined, df["LLM_KMeans_Cluster"])

# Display evaluation results
print("Silhouette Score (LLM + K-Means):", silhouette_llm)
print("Davies-Bouldin Index (LLM + K-Means):", davies_bouldin_llm)

/var/folders/4g/7cwxt52n09sb7vzyh6tzj3w00000gn/T/ipykernel_63535/1727149644.py:20: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


Silhouette Score (LLM + K-Means): 0.3194321154853902
Davies-Bouldin Index (LLM + K-Means): 1.1509411021838645


In [21]:
# Compute mean values for numerical features per cluster
cluster_means_llm = df.groupby("LLM_KMeans_Cluster")[numerical_features].mean()

# Heatmap of cluster profiles
plt.figure(figsize=(10, 6))
sns.heatmap(cluster_means_llm, annot=True, cmap="coolwarm")
plt.title("LLM + K-Means Cluster Profiles")
plt.show()

/var/folders/4g/7cwxt52n09sb7vzyh6tzj3w00000gn/T/ipykernel_63535/3542166570.py:8: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [22]:
# Get mode of text features for each cluster
cluster_text_modes = df.groupby("LLM_KMeans_Cluster")[text_columns].agg(lambda x: x.mode()[0])
print(cluster_text_modes)

                    Category Subscription Status Payment Method
LLM_KMeans_Cluster                                             
0                   Clothing                  no         PayPal
1                   Clothing                  no         PayPal
2                   Clothing                  no    Credit Card
3                   Clothing                  no    Credit Card


In [23]:
# Define LLM-based personas
llm_personas = {
    0: "Loyal Mid-Value Shopper (Low recency (recent purchases), moderate frequency, and moderate monetary spend. These customers are engaged and buy consistently but are not necessarily high spenders.)",
    1: "Super-Frequent Budget Shopper (Highest frequency (52 purchases) but lower monetary spend. These are power shoppers who make many small transactions, likely looking for deals or daily essentials.)",
    2: "Steady Mid-Spender (Mid-range frequency (25 purchases) and moderate spend, suggesting a balanced shopping behavior with steady purchasing patterns.)",
    3: "Occasional High-Value Buyer (Highest recency (long time since last purchase) but the highest monetary spend per transaction. These customers buy infrequently but spend significantly when they do.)"
}

# Assign personas
df["LLM_KMeans_Persona"] = df["LLM_KMeans_Cluster"].map(llm_personas)

# Display persona breakdown
persona_summary = df.groupby("LLM_KMeans_Cluster")[numerical_features].mean().reset_index()
persona_summary["Persona"] = persona_summary["LLM_KMeans_Cluster"].map(llm_personas)
print(persona_summary)

   LLM_KMeans_Cluster    Recency  Frequency   Monetary  Engagement_Score  \
0                   0  38.212650  11.817884  79.347874          3.791276   
1                   1  25.220923  11.569201  37.543210          3.710396   
2                   2  25.727612  52.000000  58.744403          3.756343   
3                   3  12.362335  11.596916  78.252203          3.771476   

                                             Persona  
0  Loyal Mid-Value Shopper (Low recency (recent p...  
1  Super-Frequent Budget Shopper (Highest frequen...  
2  Steady Mid-Spender (Mid-range frequency (25 pu...  
3  Occasional High-Value Buyer (Highest recency (...  


In [24]:
#Customer segmentation data
segment_data = df.groupby("LLM_KMeans_Persona")[numerical_features].mean().reset_index()
segment_data

df_segments = pd.DataFrame(segment_data)

In [25]:
# Define marketing recommendations
segment_recommendations = {
    "Loyal Mid-Value Shopper": "Targeted promotions for high-margin items to increase the average transaction value.",
    "Super-Frequent Budget Shopper": "Special discounts for frequent purchases to drive customer loyalty and retention.",
    "Steady Mid-Spender": "Personalized product recommendations based on past purchase history to increase cross-selling opportunities.",
    "Occasional High-Value Buyer": "Exclusive offers for high-value items to encourage repeat purchases and re-engage these customers."
}

# Assign recommendations
df_segments["Marketing_Recommendations"] = df_segments["LLM_KMeans_Persona"].map(segment_recommendations)

# Display segment data with recommendations
df_segments

,LLM_KMeans_Persona,Recency,Frequency,Monetary,Engagement_Score,Marketing_Recommendations
0,Loyal Mid-Value Shopper (Low recency (recent p...,38.212650,11.817884,79.347874,3.791276,NaN
1,Occasional High-Value Buyer (Highest recency (...,12.362335,11.596916,78.252203,3.771476,NaN
2,Steady Mid-Spender (Mid-range frequency (25 pu...,25.727612,52.000000,58.744403,3.756343,NaN
3,Super-Frequent Budget Shopper (Highest frequen...,25.220923,11.569201,37.543210,3.710396,NaN


In [26]:
# Streamlit Dashboard Layout
st.set_page_config(page_title="Customer Segmentation Dashboard", layout="wide")

# Dashboard Title
st.title("📊 Customer Segmentation Dashboard")

2025-03-04 02:50:49.832 
  command:

    streamlit run /opt/anaconda3/lib/python3.12/site-packages/ipykernel_launcher.py [ARGUMENTS]


DeltaGenerator()

In [29]:
# Add Customer Count column
df_segments["Customer Count"] = df.groupby("LLM_KMeans_Persona").size().values

# Key Metrics Section
st.subheader("📈 Key Business Metrics")
col1, col2, col3, col4 = st.columns(4)
col1.metric("Total Customers", sum(df_segments["Customer Count"]))
col2.metric("Avg Recency", round(df_segments["Recency"].mean(), 1))
col3.metric("Avg Frequency", round(df_segments["Frequency"].mean(), 1))
col4.metric("Predicted Revenue Growth", "$500,000 (15% uplift)")

DeltaGenerator()

In [31]:
# Pie Chart: Customer Segments Distribution
st.subheader("📌 Customer Segments Breakdown")
fig_pie = px.pie(df_segments, names="LLM_KMeans_Persona", values="Customer Count", title="Customer Segments Distribution")
st.plotly_chart(fig_pie, use_container_width=True)

DeltaGenerator()

In [33]:
# Radar Chart: Segment Comparison
st.subheader("🕵️‍♂️ Segment Comparison")
fig, ax = plt.subplots(figsize=(6, 6), subplot_kw=dict(polar=True))
categories = ["Recency", "Frequency", "Monetary", "Engagement_Score"]
angles = np.linspace(0, 2 * np.pi, len(categories), endpoint=False).tolist()

for index, row in df_segments.iterrows():
    values = row[categories].tolist()
    ax.fill(angles, values, alpha=0.25, label=row["LLM_KMeans_Persona"])
    ax.plot(angles, values, linewidth=2)

ax.set_xticks(angles)
ax.set_xticklabels(categories)
plt.legend(loc="upper right", bbox_to_anchor=(1.3, 1))
st.pyplot(fig)

DeltaGenerator()

In [34]:
# Customer Journey Insights (Simulated Line Chart)
st.subheader("📊 Customer Journey Insights")
customer_journey = {
    "Month": ["Jan", "Feb", "Mar", "Apr", "May"],
    "Loyal Shopper": [100, 120, 150, 160, 180],
    "Budget Shopper": [200, 230, 250, 280, 300],
    "Mid-Spender": [90, 100, 110, 130, 140],
    "High-Value Buyer": [50, 60, 70, 80, 90],
}
df_journey = pd.DataFrame(customer_journey)
fig_journey = px.line(df_journey, x="Month", y=df_journey.columns[1:], title="Segment Growth Over Time")
st.plotly_chart(fig_journey, use_container_width=True)


DeltaGenerator()

In [37]:
# Recommendations Section
st.subheader("📌 AI-Powered Marketing Recommendations")
selected_segment = st.selectbox("Select a Customer Segment:", df_segments["LLM_KMeans_Persona"])

# Map the selected segment to the corresponding key in segment_recommendations
segment_key = next((key for key, value in llm_personas.items() if value == selected_segment), None)

if segment_key is not None:
	st.write(f"**Recommendation:** {segment_recommendations[llm_personas[segment_key].split('(')[0].strip()]}")
else:
	st.write("**Recommendation:** No recommendation available for the selected segment.")

In [38]:
# Export Button
st.subheader("📤 Export & CRM Integration")
if st.button("Export Customer Segments"):
    df_segments.to_csv("customer_segments.csv", index=False)
    st.success("✅ Customer Segments exported successfully!")

st.write("🔗 **Integration Available**: Connect with CRM tools like Salesforce, HubSpot, and Klaviyo.")